# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        #print(row)
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

101


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
data=pd.read_csv('event_datafile_new.csv')
print(data)
from cassandra.cluster import Cluster
cluster = Cluster()
# To establish connection and begin executing queries, need a session
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
        print(e)



                             artist firstName gender  itemInSession  lastName  \
0                              Mynt   Celeste      F              2  Williams   
1                      Taylor Swift   Celeste      F              3  Williams   
2                     Amy Winehouse   Celeste      F              4  Williams   
3                   Jimmy Eat World   Celeste      F              5  Williams   
4                     Maldita Nerea  Anabelle      F              0   Simpson   
5                             Fluke    Connar      M              0    Moreno   
6                    Habib KoitÃÂ©    Jayden      M              0       Fox   
7                         The Kooks      Sara      F              0   Johnson   
8                    Blues Traveler      Sara      F              1   Johnson   
9                          Coldplay      Sara      F              2   Johnson   
10  Tom Petty And The Heartbreakers      Sara      F              3   Johnson   
11                        Gi

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 

try:
    session.execute(
    "CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")    
except Exception as e:
        print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
        print(e)
   

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### FUNCTIONS
#### There are three functions for creating tables,inserting data and for select queries to verify if the data is properly inserted into tables.


In [9]:

def create_table(table_name):
    ''' This will create the table with specified table name 
         and adds the columns which are required for that table from the
         csv file.'''
    
    query = "CREATE TABLE IF NOT EXISTS "+table_name
    if(table_name=="session_history"):
        query = query + "( sessionId int, itemInSession int, artist text, song text,length double, PRIMARY KEY (sessionId,itemInSession))"

    elif(table_name=="user_history"):
        query= query + "( userId int,firstName text,lastName text,artist text,song text,itemInSession int,sessionId int,PRIMARY KEY (userId,sessionId,itemInSession));"
    
    elif(table_name=="song_history"):
        query= query + "(userId int,firstName text,lastName text,song text,PRIMARY KEY (song,userId));"

    try:
        session.execute(query)
    except Exception as e:
        print(e)
        
def insert_table(table_name):
    ''' Depending upon which table name is passed this will create insert 
        query,then pass the corresponding columns to the insert statement to
        insert the records into the table.'''
    
    ## TO-DO: Assign the INSERT statements into the `query` variable depending on table name

    query=""
    file="event_datafile_new.csv"
    if(table_name=="session_history"):
        query = "INSERT INTO session_history (sessionId,itemInSession,artist,song,length)"
        query = query + " VALUES (%s, %s, %s, %s,%s);"
    elif (table_name=="user_history"):
        query = "INSERT INTO user_history ( userId,firstName,lastName,artist,song,itemInSession,sessionId)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s);"
    elif(table_name=="song_history"):
        query="INSERT INTO song_history (userId,firstName,lastName,song)"
        query = query + " VALUES (%s, %s, %s, %s)"
    
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
    
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
            if(table_name=="session_history"):
                session.execute(query, (int(line[8]),int(line[3]),str(line[0]),str(line[9]),float(line[5])))
            elif(table_name=="user_history"):
                session.execute(query, (int(line[10]),line[1],line[4],str(line[0]),line[9],int(line[3]),int(line[8])))
            elif(table_name=="song_history"):
                session.execute(query, (int(line[10]),line[1],line[4],str(line[9])))
                        
def verify_select(table_name):
    ''' This function is used to select all rows from the table
    to verify if the data is properly inserted.'''
    
    ## TO-DO: Add in the SELECT statement to verify the data was entered into the table
    query="select * from "+ table_name 

    df=pd.DataFrame()
    try:
        df = pd.DataFrame(list(session.execute(query)))
    except Exception as e:
        print(e)

    if(df.empty):
        print("No records present in the table"+ table_name)
    else:
        print(df)



### SESSION_HISTORY TABLE
##### 1. For first query session_history table is created which has columns sessionId,itemInsession,artist name,length,song title. sessionId is used as Partition Key and itemSession is used as Clustering key and it has been used in select statement in the same order.

##### 2. As there are no matching records, query does not return anything.

In [24]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
#Create the table with the specified name for the first query, using the create_table 
#function.

create_table("session_history")

#insert data into session_history table.
insert_table("session_history")

#Do a SELECT to verify that the data have been inserted into each table
#Verify whether data is properly inserted.
verify_select("session_history")

## Write the specified query.

query = "select * from session_history WHERE sessionId = 338 and itemInSession = 5"

#Use pandas dataframe to display the results.
df=pd.DataFrame()
try:
    df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)

if df.empty :
    print("No records found for given session!")
else:
    print(df)

    sessionid  iteminsession                           artist     length  \
0         185              0                   Habib KoitÃÂ©  285.15220   
1           5              0                        Sugarland  247.77098   
2           5              1                  A Hope For Home  388.38812   
3         210              1                     Klaus Badelt  128.62649   
4          52              2                             Mynt  166.94812   
5          52              3                     Taylor Swift  230.47791   
6          52              4                    Amy Winehouse  229.85098   
7          52              5                  Jimmy Eat World  285.83138   
8         152              0                        The Kooks  132.25751   
9         152              1                   Blues Traveler  290.24608   
10        152              2                         Coldplay  298.76200   
11        152              3  Tom Petty And The Heartbreakers  183.01342   
12        15

## Same code is repeated for remaining two queries.

### USER_HISTORY TABLE
##### 1.For second query user_history table is created which has columns  firstName,lastName,artist,song,sessionId,itemInsession,artist name. userId is used as Partition Key and sessionId and itemInSession are  used as Clustering key and it has been used in select statement in the same order. As itemInSession is clustering column records are sorted in desc order of itemInSession.

##### 2. Implemented second query here but  there are no records matching given criteria.

In [25]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_table("user_history")
insert_table("user_history")
verify_select("user_history")

query = "select artist,song,firstName,lastName from user_history WHERE userId = 10 and sessionId =182"
df=pd.DataFrame()
try:
    df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)

if df.empty :
    print("No records found for given session!")
else:
    print(df)

    userid  sessionid  iteminsession                           artist  \
0       53         52              2                             Mynt   
1       53         52              3                     Taylor Swift   
2       53         52              4                    Amy Winehouse   
3       53         52              5                  Jimmy Eat World   
4       95        152              0                        The Kooks   
5       95        152              1                   Blues Traveler   
6       95        152              2                         Coldplay   
7       95        152              3  Tom Petty And The Heartbreakers   
8       95        152              4                        Girl Talk   
9       95        152              5           Florence + The Machine   
10      95        152              6                     Three Drives   
11      95        152              7                   Jonas Brothers   
12      95        152              8               


### SONG_HISTORY TABLE
##### 1.For third query song_history table is created which has columns  userId,firstName,lastName,song title. userId is used as Partition Key and song is used as Clustering key. Using userId as partition key ensures that it every row is unique even if there are duplicated user names. Records are sorted on song title.

##### 2. Implemented third query and one record has been found matching given criteria.

In [29]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_table("song_history")
insert_table("song_history")

#Do a SELECT to verify that the data have been inserted into each table
verify_select("song_history")

query = "select firstName,lastName from song_history WHERE song='All Hands Against His Own'"
df=pd.DataFrame()
try:
    df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)

if df.empty :
    print("No records found for given session!")
else:
    print(df)

                    

                                                 song  userid firstname  \
0               Slave To Love (1999 Digital Remaster)      15      Lily   
1                                         Greece 2000      95      Sara   
2   Catch You Baby (Steve Pitron & Max Sanna Radio...      15      Lily   
3                     Round And Round (Soul Mix Edit)      95      Sara   
4                          Rabbit Heart (Raise It Up)      15      Lily   
5                                           Wasteland      15      Lily   
6                                             Florida      95      Sara   
7                                             Bermuda      62    Connar   
8                                             Ray Gun      95      Sara   
9                                  You Belong With Me      53   Celeste   
10                                           If I Was      95      Sara   
11                                           For Emma      95      Sara   
12                       

### Drop the tables before closing out the sessions

In [27]:
## TO-DO: Drop the table before closing out the sessions

In [10]:
query = "drop table IF EXISTS session_history"
query1 = "drop table IF EXISTS user_history"
query2 = "drop table IF EXISTS song_history"



try:
    rows = session.execute(query)
    rows1 = session.execute(query1)
    rows2 = session.execute(query2)


except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [11]:
session.shutdown()
cluster.shutdown()